In [22]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import *


In [ ]:
def normalization(data):
    min_vals = np.min(data, axis=0)
    max_vals = np.max(data, axis=0)
    _range = max_vals - min_vals
    return (data - min_vals) / _range

original_data = pd.read_csv('./datasets/AEEEM/ML.csv')       

original_data.isnull().values.any()  # Gives false ie:No null value in dataset
original_data = original_data.fillna(value=False) 
original_Y = original_data['class']  
original_Y = pd.DataFrame(original_Y)    
original_data = normalization(original_data)    

original_X = pd.DataFrame(original_data.drop(['class'], axis=1))  
 
x_train, x_test, y_train, y_test = train_test_split(original_X, original_Y, test_size=.1, random_state=12)
print(x_train.shape, y_train.shape,x_test.shape, y_test.shape)

sm = SMOTE(random_state=12, sampling_strategy=1.0)  
x, y = sm.fit_resample(x_train, y_train)  
y_train = pd.DataFrame(y, columns=['class'])    
x_train = pd.DataFrame(x, columns=original_X.columns)

(1675, 21) (1675, 1) (187, 21) (187, 1)


In [ ]:
#x_test.loc[1,'total_loc'] =  0.264528801 
#x_test.loc[1,'comment_loc'] =  0.337140831 
#x_test.loc[1,'executable_loc'] =  0.203275228 
#x_test.loc[1,'unique_operands'] =  0.369565217 
#x_test.loc[1,'unique_operators'] =  0.479770399 
#x_test.loc[1,'total_operands'] =  0.282879173 
#x_test.loc[1,'total_operators'] =  0.250517298 
#x_test.loc[1,'halstead_vocabulary'] =  0.414459875 
#x_test.loc[1,'halstead_length'] =  0.262842023 
#x_test.loc[1,'halstead_volume'] =  0.244372278 
#x_test.loc[1,'halstead_level'] =  0.500382013 
#x_test.loc[1,'halstead_difficulty'] =  0.280770116 
#x_test.loc[1,'halstead_effort'] =  0.152186111 
#x_test.loc[1,'halstead_time'] =  0.152185296 
#x_test.loc[1,'branch_count'] =  0.172317183 
#x_test.loc[1,'condition_count'] =  0.171775777 
#x_test.loc[1,'cyclomatic_complexity'] =  0.179914274 
#x_test.loc[1,'cyclomatic_density'] =  0.646325185 
#x_test.loc[1,'decision_density'] =  0.290946038 
#x_test.loc[1,'design_density'] =  0.200977818 
#x_test.loc[1,'normalized_cyclomatic_complexity'] =  0.455357667 

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.1, random_state=12)

x_train = x_train.values
x_val = x_val.values
x_test = x_test.values
y_train = y_train.values
y_val = y_val.values
y_test = y_test.values

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)


(2602, 21, 1) (2602, 1) (290, 21, 1) (290, 1) (187, 21, 1) (187, 1)


In [ ]:
n_classes = 2

y_train = tf.keras.utils.to_categorical(y_train, n_classes)
y_val = tf.keras.utils.to_categorical(y_val, n_classes)
y_test = tf.keras.utils.to_categorical(y_test, n_classes)

print(y_train.shape, y_val.shape, y_test.shape)

(2602, 2) (290, 2) (187, 2)


In [ ]:
x_train = x_train.astype(np.float32)
x_val = x_val.astype(np.float32)
x_test = x_test.astype(np.float32)


In [ ]:
input_shape  = (x_train.shape[1], 1)

# print(f'input_shape: {input_shape}')

input_shape: (21, 1)


In [ ]:
x_train_flat = x_train.reshape((x_train.shape[0], -1))
x_val_flat = x_val.reshape((x_val.shape[0], -1))
x_test_flat = x_test.reshape((x_test.shape[0], -1))

# x_train_flat.shape, x_val_flat.shape, x_test_flat.shape

In [30]:
n_features = len(x_train_flat[1])
n_features

21

In [ ]:
from models.convnet import ConvNet
import glob

batch_size = 16
epochs = 40 

'''If you wish to train your own model instead of loading one from checkpoint, remove the checkpoint.'''
# for f in glob.glob('assets/teacher_model.hdf5'):
#    os.remove(f)

nn = ConvNet(input_shape, n_classes=n_classes)   
 
nn.maybe_train(data_train=(x_train, y_train),  
               data_valid=(x_val, y_val),  
               batch_size=batch_size , epochs=epochs)  

nn.evaluate(x_val, y_val)
nn.evaluate(x_test, y_test)

No checkpoint found on assets/teacher_model.hdf5. Training from scratch.
Epoch 1/40
82/82 [==============================] - 2s 30ms/step - loss: 0.6479 - acc: 0.6084 - val_loss: 0.6945 - val_acc: 0.5000
Epoch 2/40
82/82 [==============================] - 2s 23ms/step - loss: 0.6085 - acc: 0.6387 - val_loss: 0.6892 - val_acc: 0.5310
Epoch 3/40
82/82 [==============================] - 2s 28ms/step - loss: 0.5991 - acc: 0.6603 - val_loss: 0.6943 - val_acc: 0.4621
Epoch 4/40
82/82 [==============================] - 2s 26ms/step - loss: 0.5957 - acc: 0.6618 - val_loss: 0.6892 - val_acc: 0.5000
Epoch 5/40
82/82 [==============================] - 2s 25ms/step - loss: 0.5867 - acc: 0.6683 - val_loss: 0.7553 - val_acc: 0.5000
Epoch 6/40
82/82 [==============================] - 2s 28ms/step - loss: 0.5757 - acc: 0.6695 - val_loss: 0.7022 - val_acc: 0.4966
Epoch 7/40
82/82 [==============================] - 2s 26ms/step - loss: 0.5630 - acc: 0.6918 - val_loss: 0.7346 - val_acc: 0.5000
Epoch 8/40

In [32]:
y_train_soft = nn.predict(x_train)

In [ ]:
from models.tree import SoftDecisionTree
from tensorflow.keras.optimizers import Adam

learning_rate = 3e-4  
max_depth = 4
n_features = n_features
n_classes = 2
penalty_strength = 1e+1
penalty_decay = 0.25
inv_temp = 0.01  
epochs = 40
ema_win_size = 100
batch_size = 16

optimizer = Adam(lr=learning_rate)

tree_model = SoftDecisionTree(max_depth=max_depth, n_features=n_features, n_classes=n_classes, 
                          penalty_strength=penalty_strength, penalty_decay=penalty_decay,  
                          inv_temp=inv_temp, ema_win_size=ema_win_size) 
 
data_train = x_train_flat, y_train  
data_val =  x_val_flat, y_val  

tree_model.tree_train(x_train_flat, y_train_soft, x_val_flat, y_val, batch_size, epochs , distill=True) 


No checkpoint found at assets/distilled/. Training from scratch.
Starting training...
Epoch 1/40
163/163 [==============================] - 1s 6ms/step - loss: 5.5323 - accuracy: 0.5719 - val_loss: 5.5106 - val_accuracy: 0.5000
Epoch 2/40
163/163 [==============================] - 0s 3ms/step - loss: 5.4630 - accuracy: 0.5907 - val_loss: 5.4720 - val_accuracy: 0.5517
Epoch 3/40
163/163 [==============================] - 0s 3ms/step - loss: 5.4157 - accuracy: 0.6722 - val_loss: 5.4419 - val_accuracy: 0.6103
Epoch 4/40
163/163 [==============================] - 0s 3ms/step - loss: 5.3753 - accuracy: 0.7018 - val_loss: 5.4150 - val_accuracy: 0.6207
Epoch 5/40
163/163 [==============================] - 1s 3ms/step - loss: 5.3399 - accuracy: 0.7244 - val_loss: 5.3911 - val_accuracy: 0.6483
Epoch 6/40
163/163 [==============================] - 1s 3ms/step - loss: 5.3079 - accuracy: 0.7375 - val_loss: 5.3689 - val_accuracy: 0.6483
Epoch 7/40
163/163 [==============================] - 1s 4ms/s

In [ ]:
loss   = 12.62952525799091   #53.85
loss1  = 12.629525331350473
loss2  = 12.629525184631348
loss3  = 12.629525111271786
loss4  = 12.629525184631348
loss5  = 12.629525331350473
loss6  = 12.629525184631348
loss7  = 12.629525331350473
loss8  = 12.629525331350473
loss9  = 12.62952525799091
loss10 = 12.62952525799091
loss11 = 12.629525184631348
loss12 = 12.629525184631348
loss13 = 12.629525331350473
loss14 = 12.629525331350473
loss15 = 12.629525404710035
loss16 = 12.62952525799091
loss17 = 12.62952525799091
loss18 = 12.629525184631348
loss19 = 12.629525331350473    #46.15% 
loss20 = 12.629525184631348
loss21 = 12.62952525799091

Sensitivity1 = (loss1 - loss) / 0.21076536
Sensitivity2 = (loss2 - loss) / 0.203807498
Sensitivity3 = (loss3 - loss) / 0.190775228
Sensitivity4 = (loss4 - loss) / 0.217391304
Sensitivity5 = (loss5 - loss) / 0.244476282
Sensitivity6 = (loss6 - loss) / 0.205956096
Sensitivity7 = (loss7 - loss) / 0.200793541
Sensitivity8 = (loss8 - loss) / 0.228019197
Sensitivity9 = (loss9 - loss) / 0.202439338
Sensitivity10 = (loss10 - loss) / 0.199596158
Sensitivity11 = (loss11 - loss) / 0.252959332
Sensitivity12 = (loss12 - loss) / 0.197256351
Sensitivity13 = (loss13 - loss) / 0.146958898
Sensitivity14 = (loss14 - loss) / 0.146958831
Sensitivity15 = (loss15 - loss) / 0.172317183
Sensitivity16 = (loss16 - loss) / 0.171775777
Sensitivity17 = (loss17 - loss) / 0.179914274
Sensitivity18 = (loss18 - loss) / 0.22079327
Sensitivity19 = (loss19 - loss) / 0.290946038
Sensitivity20 = (loss20 - loss) / 0.200977818
Sensitivity21 = (loss21 - loss) / 0.221315114

print(Sensitivity1)
print(Sensitivity2)
print(Sensitivity3)
print(Sensitivity4)
print(Sensitivity5)
print(Sensitivity6)
print(Sensitivity7)
print(Sensitivity8)
print(Sensitivity9)
print(Sensitivity10)
print(Sensitivity11)
print(Sensitivity12)
print(Sensitivity13)
print(Sensitivity14)
print(Sensitivity15)
print(Sensitivity16)
print(Sensitivity17)
print(Sensitivity18)
print(Sensitivity19)
print(Sensitivity20)
print(Sensitivity21)